In [3]:
:dep nalgebra
:dep replace_with
:dep plotly

# Base
   
o codigo abaixo serve para criar uma estrutura base para trabalharmos nos algoritmos de otimização.

In [8]:
use std::iter;
use nalgebra::DMatrix;
use replace_with::replace_with_or_abort_and_return;

#[derive(Clone, Copy, PartialEq, Eq, Debug,Hash)]
pub struct Passo<X, Y> {
	pub x: X,
	pub y: Y
}

/// Um trait é uma funcionalidade que algo pode oferecer. 
/// Podemos usa-lo no lugar de tipos pre construidos para criar algoritmos genericos
/// No caso, um objeto implementa AlgoritmoOtimizacao se ele é capaz de calcular 
/// um novo passo de otimização a partir de um antigo

pub trait AlgoritmoOtimizacao<X,Y>{
	fn proximo_passo(&mut self, passo: &Passo<X, Y>) -> Passo<X, Y>;
}
// Um criterio de parada 
pub trait CriterioParada<X, Y>{
	fn deve_parar(&self, passo_anterior: &Passo<X, Y> , passo_seguinte: &Passo<X, Y>) -> bool;
}

// função que monta a execucao. ela recebe o algoritmo, um criterio de parada e um estado inicial
pub fn monte_otimizacao<X, Y, Algo, Crit>
	(mut algo: Algo, crit: Crit, passo_inicial: Passo<X, Y>) -> impl Iterator<Item=Passo<X, Y>> 
	where 
		Algo: AlgoritmoOtimizacao<X,Y>,
		Crit: CriterioParada<X, Y>
{
	#[derive(Clone, PartialEq, Eq)]
	enum EstadoExecOtimizacao<X, Y>{
		Executando(Passo<X,Y>),
		Finalizando(Passo<X,Y>),
		Acabado,
	}
    
	let mut estado_exec = EstadoExecOtimizacao::Executando(passo_inicial);
	
	iter::from_fn(move || replace_with_or_abort_and_return(&mut estado_exec, |estado_exec: EstadoExecOtimizacao<X, Y>| 
		match estado_exec{
			EstadoExecOtimizacao::Executando(passo_guardado) => {
				let novo_passo = algo.proximo_passo(&passo_guardado);  //calculamos o proximo
				let proximo_estado_exec = if crit.deve_parar(&passo_guardado, &novo_passo){
					EstadoExecOtimizacao::Finalizando(novo_passo)
				} else {
					EstadoExecOtimizacao::Executando(novo_passo)
				};
				(Some(passo_guardado), proximo_estado_exec)
			}
			EstadoExecOtimizacao::Finalizando(passo_guardado) => (Some(passo_guardado), EstadoExecOtimizacao::Acabado),
			EstadoExecOtimizacao::Acabado => (None, EstadoExecOtimizacao::Acabado),
		}
	))
}

#### Criamos um criterio de parada simples

In [9]:
pub struct CriterioDeltaXMenorQueEpsilon(pub f64);
impl CriterioParada<f64, f64> for CriterioDeltaXMenorQueEpsilon		{
	fn deve_parar(&self, passo_anterior: &Passo<f64, f64> , passo_seguinte: &Passo<f64, f64>) -> bool {
		(passo_anterior.x - passo_seguinte.x).abs() <= self.0
	}
}
impl CriterioParada<Vetor, f64> for CriterioDeltaXMenorQueEpsilon{
	fn deve_parar(&self, passo_anterior: &Passo<Vetor, f64> , passo_seguinte: &Passo<Vetor, f64>) -> bool {
		let delta_x = passo_seguinte.x.clone() - passo_anterior.x.clone();
		delta_x.norm() <= self.0
	}
}


#### E alguns apelidos, para ajudar com a legibilidade

In [11]:
pub type PassoEscalar = Passo<f64, f64>;
pub type PassoVet = Passo<Vetor, f64>;   

// apelido para esses tipos, para facilitar a leitura.
pub type Vetor = nalgebra::DVector<f64>;  
pub type Matriz = nalgebra::DMatrix<f64>; 

# Ferramentas

In [16]:
pub mod ferramentas{
    pub mod calculo{
        // conjunto de funcionalidades 
        pub mod linear_func{
            #[derive(PartialEq, Clone)]
            pub struct LinearFunc{
                pub a: f64,
                pub b: f64,
            }

            impl LinearFunc{
                pub fn calcule(&self, x: &f64) -> f64{
                    self.a * x + self.b
                }
                pub fn cruzamento_com_eixo_x(&self) -> f64{
                    -self.b / self.a
                }
                pub fn cruzamento_com_eixo_y(&self) -> f64{
                    todo!()				
                }
                pub fn novo_de_ponto_e_derivada(point: (&f64, &f64), deriv: &f64) -> LinearFunc{
                    let a = *deriv;	
                    let b = -(a * point.0) + point.1;
                    LinearFunc{a , b}
                }
            }

        }
        
		pub mod diff{
            use nalgebra::Vector;
    

			pub mod escalar{
				pub fn derivada_numerica(delta: f64, mut f: impl Fn(&f64) -> f64 + Clone) -> impl (Fn(&f64) -> f64) + Clone{
					move |x|{
						let x_plus_delta = x + delta/2.0;
						let x_minus_delta  = x - delta/2.0;
						let y_of_plus_delta = f(&x_plus_delta);
						let y_of_minus_delta = f(&x_minus_delta);
						let df = (y_of_plus_delta - y_of_minus_delta)/delta; 
						df
					}
				}
			}
		
		
			pub mod vetorial{
				use nalgebra::{Const, SMatrix};
				use crate::Vetor;

					
					/// Essa função recebe uma função e um parametro delta e retorna uma outra função a qual retorna uma estimativa da derivada no ponto. Essa estimativa é calculada numericamente.
					pub fn derivada_numerica
						(delta: f64, mut f: impl Fn(&Vetor) -> f64) -> impl Fn(&Vetor) -> Vetor
					{ 
						move |x| {
							let x_dim = x.len();
							let mut ans = Vetor::zeros(x_dim);
							for ix in 0..x_dim {
								let delta_vec = {
									let mut delta_vec = Vetor::zeros(x_dim);
									delta_vec[ix] = delta;
									delta_vec
								};
								let x_mais_delta    = x + delta_vec.clone();
								let x_menos_delta   = x - delta_vec.clone();

								let y_de_mais_delta = f(&x_mais_delta); 
								let y_de_menos_delta = f(&x_menos_delta); 

								let delta_y = y_de_mais_delta - y_de_menos_delta; 
								let df = delta_y / (delta * 2.0);
								ans[ix] = df;
							}
							ans
						}
					}
			}

			pub mod matriz{
				use nalgebra::SMatrix;
                use crate::{Matriz, Vetor};
				

				pub fn derivada_numerica
					(   delta: f64,
						mut f: impl Fn(&Vetor) -> Vetor
					) 
					-> impl Fn(&Vetor) -> Matriz
				{
					move |x| {
						let in_dim  = x.len();
						let out_dim = (f(x)).len(); 

						let mut ans = Matriz::zeros(in_dim, out_dim);
						for ix in 0..in_dim{
							let delta_vec = {
								let mut delta_vec = Vetor::zeros(in_dim);
								delta_vec[ix] = delta  ;
								delta_vec
							};
							let x_mais_delta    = x + delta_vec.clone();
							let x_menos_delta   = x - delta_vec.clone();

							let y_de_mais_delta = f(&x_mais_delta); 
							let y_de_menos_delta = f(&x_menos_delta); 

							let delta_y = y_de_mais_delta - y_de_menos_delta; 
							let df = delta_y / (delta * 2.0);
							for iy in 0..out_dim{
								ans[iy * in_dim + ix] = df[iy]
							}
						}
						ans
					}		
				}

			}

			#[test]
			fn teste_derivada_matriz(){

				let f= |v: &Vetor| -> Vetor{
					let x1 = v[0].clone();
					let x2 = v[1].clone();
					let y1 = x1.powi(2) * 3.0 + -2.0 * x1 + 10.0*x2 + 2.0*x1*x2;
					let y2 = x1 + -2.0 * x1 + 10.0*x2 + 2.0*x1*x2;
					let y3 = x2*1.2;

					Vetor::from_row_slice(&[y1, y2, y3])
				};

				let delta = 10e-4;
				let df = matriz::derivada_numerica(delta, &f); 

				let exemplo_x = Vetor::zeros(2);
				let exemplo_y = df(&exemplo_x);
				println!("{:?}", &exemplo_y);

			}
		}
	}

	pub mod graficos{
        use std::ops::Range;
		pub fn linspace(range: Range<f64>, num_pontos: u32) -> impl ExactSizeIterator<Item=f64> + Clone{
			(0..num_pontos)
			.into_iter()
			.map({
				let passo = (range.end - range.start) / ((num_pontos - 1) as f64);
				move |n| {
					return passo * (n as f64) + range.start;
				}
			})
		}
	}

	pub mod exemplos{
		pub mod escalar{
            use crate::PassoEscalar;
			pub fn obtenha_f_e_passo_inicial() -> (impl Fn(&f64) -> f64, PassoEscalar){
				let f = |x:&f64| x.powf(2.0) + 3.0*x + 4.0 ;

				let x_inicial = 15.0;
				let y_inicial = f(&x_inicial);
				let passo_inicial = PassoEscalar{x: x_inicial, y: y_inicial};

				(f, passo_inicial)
			}
		}
		pub mod vetorial{ 
			use crate::{Passo, PassoVet, Vetor};
			pub fn obtenha_f_e_passo_inicial() -> (impl Fn(&Vetor) -> f64 + Clone, PassoVet){
				let f = |x: &Vetor| -> f64{
					let x_0 = x[0]; 
					let x_1 = x[1];
					x_0.powf(2.0) + x_0*3.0 + 4.0 + x_1.powi(2)*2.0 
				};

				let x_inicial = Vetor::from_row_slice(&[10.0, 15.0]);
				let y_inicial = f(&x_inicial);
				let passo_inicial = Passo{
					x: x_inicial, 
					y: y_inicial
				};

				(f, passo_inicial)
			}
		}
	}
}


 Agora que temos algumas coisas escritas, podemos trabalhar com os algoritmos
 

# Algoritmos

## Descida de Gradiente

### Definição

In [20]:
pub mod descida_grad{
    use crate::{ferramentas::{calculo::{self, diff}, exemplos}, monte_otimizacao, AlgoritmoOtimizacao, CriterioDeltaXMenorQueEpsilon, Passo, PassoEscalar, Vetor};

    pub struct DescidaGrad<F, Df>{
        pub f : F,
        pub df : Df,
        pub alpha: f64,
    }


    impl<F, Df> AlgoritmoOtimizacao<f64, f64> for DescidaGrad<F, Df> 
        where F: Fn(&f64) -> f64, Df: Fn(&f64) -> f64
    {
        fn proximo_passo(&mut self, passo_anterior: &Passo<f64, f64>) -> Passo<f64, f64> {
            let novo_x = passo_anterior.x - (self.alpha *  (self.df)(&passo_anterior.x));
            let novo_y = (self.f)(&novo_x);
            Passo{x: novo_x, y: novo_y}
        }
    }

    impl<F, Df> AlgoritmoOtimizacao<Vetor, f64> for DescidaGrad<F, Df> where F: Fn(&Vetor) -> f64, Df: Fn(&Vetor) -> Vetor{
        fn proximo_passo(&mut self, passo_anterior: &Passo<Vetor, f64>) -> Passo<Vetor, f64> {
            let novo_x = passo_anterior.x.clone() - (self.alpha * (self.df)(&passo_anterior.x));
            let novo_y = (self.f)(&novo_x);
            Passo{x: novo_x, y: novo_y}
        }
    }
}
    

### Exemplos

#### No $\mathbb{R}^1$ (Escalar)

In [26]:
{
    use crate::ferramentas::*;
    use descida_grad::*;
    let epsilon = 10e-5;
    let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);
    let f_exemplo = |x:&f64| x.powf(2.0) + 3.0*x + 4.0 ;
    let df_exemplo = calculo::diff::escalar::derivada_numerica(10e-5, f_exemplo );
    let mut algoritmo = DescidaGrad{
        f: f_exemplo.clone(), 
        df: df_exemplo,
        alpha: 0.01,	
    };
    let x_inicial = 15.0;
    let y_inicial = f_exemplo(&x_inicial);
    let passo_inicial = PassoEscalar{x: x_inicial, y: y_inicial};
    let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
    otimizacao.enumerate().for_each(|(n_passo, passo)| {println!("{:000} - {:?}", n_passo, passo); });


    let epsilon = 10e-5;
    let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

    let (f_exemplo, passo_inicial) = crate::ferramentas::exemplos::vetorial::obtenha_f_e_passo_inicial();

    let df_exemplo = calculo::diff::vetorial::derivada_numerica(10e-6, &f_exemplo);
    let mut algoritmo = DescidaGrad{
        f: &f_exemplo, 
        df: &df_exemplo,
        alpha: 0.1,	
    };
    let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
    otimizacao.enumerate().for_each(|(n_passo, passo)| {println!("{:000} - {:?}", n_passo, passo); });

}

0 - Passo { x: 15.0, y: 274.0 }
1 - Passo { x: 14.669999999997572, y: 263.21889999992146 }
2 - Passo { x: 14.346600000001217, y: 252.8647315600386 }
3 - Passo { x: 14.0296680000057, y: 242.92058819040102 }
4 - Passo { x: 13.719074640010263, y: 233.3702328982035 }
5 - Passo { x: 13.414693147213939, y: 224.1980716755504 }
6 - Passo { x: 13.116399284273825, y: 215.3891280373204 }
7 - Passo { x: 12.824071298589388, y: 206.92901856707226 }
8 - Passo { x: 12.537589872618469, y: 198.8039294318406 }
9 - Passo { x: 12.256838075168162, y: 191.00059382639645 }
10 - Passo { x: 11.981701313666235, y: 183.50627031090988 }
11 - Passo { x: 11.712067287396053, y: 176.3087220066809 }
12 - Passo { x: 11.447825941652354, y: 169.39619661532566 }
13 - Passo { x: 11.188869422820744, y: 162.75740722939526 }
14 - Passo { x: 10.935092034366676, y: 156.38151390316955 }
15 - Passo { x: 10.686390193682769, y: 150.25810595268754 }
16 - Passo { x: 10.442662389811517, y: 144.37718495701853 }
17 - Passo { x: 10.203809

139 - Passo { x: -0.504804358195301, y: 2.7404143654670667 }
140 - Passo { x: -0.5247082710314288, y: 2.701193956594505 }
141 - Passo { x: -0.5442141056107985, y: 2.663526675913366 }
142 - Passo { x: -0.5633298234985951, y: 2.627351019547173 }
143 - Passo { x: -0.5820632270286641, y: 2.59260791917303 }
144 - Passo { x: -0.6004219624880758, y: 2.559240645573805 }
145 - Passo { x: -0.6184135232383614, y: 2.5271947160089994 }
146 - Passo { x: -0.6360452527736005, y: 2.4964178052550317 }
147 - Passo { x: -0.6533243477181561, y: 2.466859660166886 }
148 - Passo { x: -0.6702578607638365, y: 2.438472017624205 }
149 - Passo { x: -0.6868527035485705, y: 2.411208525726269 }
150 - Passo { x: -0.703115649477617, y: 2.38502466810748 }
151 - Passo { x: -0.7190533364881091, y: 2.3598776912503547 }
152 - Passo { x: -0.7346722697583541, y: 2.3357265346768297 }
153 - Passo { x: -0.7499788243632199, y: 2.3125317639035776 }
154 - Passo { x: -0.7649792478759654, y: 2.2902555060529814 }
155 - Passo { x: -0.7

()

223 - Passo { x: -1.3176529156031425, y: 1.783250459188035 }
224 - Passo { x: -1.3212998572910717, y: 1.7819337410041913 }
225 - Passo { x: -1.324873860145317, y: 1.7806691648604023 }
226 - Passo { x: -1.3283763829424888, y: 1.7794546659319033 }
227 - Passo { x: -1.3318088552836915, y: 1.7782882611609825 }
228 - Passo { x: -1.335172678178056, y: 1.7771680460189945 }
229 - Passo { x: -1.3384692246145935, y: 1.7760921913966108 }
230 - Passo { x: -1.34169984012237, y: 1.7750589406172832 }
231 - Passo { x: -1.3448658433199778, y: 1.7740666065688218 }
232 - Passo { x: -1.3479685264536378, y: 1.7731135689486779 }
233 - Passo { x: -1.351009155924645, y: 1.7721982716182865 }
234 - Passo { x: -1.3539889728062438, y: 1.7713192200621757 }
235 - Passo { x: -1.3569091933501998, y: 1.7704749789476906 }
236 - Passo { x: -1.3597710094832482, y: 1.7696641697813473 }
237 - Passo { x: -1.3625755892936375, y: 1.7688854686579907 }
238 - Passo { x: -1.3653240775078146, y: 1.7681376040991212 }
239 - Passo { 

357 - Passo { x: -1.4878327350176601, y: 1.7501480423371505 }
358 - Passo { x: -1.4880760803173487, y: 1.7501421798605983 }
359 - Passo { x: -1.488314558711048, y: 1.7501365495381176 }
360 - Passo { x: -1.4885482675368955, y: 1.7501311421764063 }
361 - Passo { x: -1.488777302186186, y: 1.75012594894622 }
362 - Passo { x: -1.4890017561424962, y: 1.7501209613679491 }
363 - Passo { x: -1.4892217210196979, y: 1.750116171297777 }
364 - Passo { x: -1.489437286599351, y: 1.7501115709143842 }
365 - Passo { x: -1.4896485408674298, y: 1.7501071527061733 }
366 - Passo { x: -1.489855570050116, y: 1.7501029094590081 }
367 - Passo { x: -1.4900584586491483, y: 1.7500988342444308 }
368 - Passo { x: -1.4902572894761956, y: 1.7500949204083507 }
369 - Passo { x: -1.4904521436866958, y: 1.7500911615601793 }
370 - Passo { x: -1.490643100812985, y: 1.750087551562396 }
371 - Passo { x: -1.490830238796761, y: 1.7500840845205246 }
372 - Passo { x: -1.491013634020879, y: 1.7500807547735109 }
373 - Passo { x: -1

41 - Passo { x: VecStorage { data: [-1.4987771102637026, 1.2029808323103445e-8], nrows: Dyn(2), ncols: Const }, y: 1.7500014954593073 }
42 - Passo { x: VecStorage { data: [-1.4990216882149277, 7.2181017343780165e-9], nrows: Dyn(2), ncols: Const }, y: 1.750000957093949 }
43 - Passo { x: VecStorage { data: [-1.4992173505767958, 4.330411647327984e-9], nrows: Dyn(2), ncols: Const }, y: 1.7500006125401195 }
44 - Passo { x: VecStorage { data: [-1.4993738804658463, 2.59846372891274e-9], nrows: Dyn(2), ncols: Const }, y: 1.750000392025671 }
45 - Passo { x: VecStorage { data: [-1.4994991043770867, 1.5592949778635938e-9], nrows: Dyn(2), ncols: Const }, y: 1.750000250896425 }
46 - Passo { x: VecStorage { data: [-1.499599283506523, 9.353496380242559e-10], nrows: Dyn(2), ncols: Const }, y: 1.750000160573708 }
47 - Passo { x: VecStorage { data: [-1.499679426809628, 5.612044787255781e-10], nrows: Dyn(2), ncols: Const }, y: 1.7500001027671703 }


#### No $\mathbb{R}^n$ para $n>1$ (Escalar)

In [28]:
{
    use crate::ferramentas::*;
    use descida_grad::*;
    
    let epsilon = 10e-5;
    let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

    let (f_exemplo, passo_inicial) = exemplos::vetorial::obtenha_f_e_passo_inicial();

    let df_exemplo = calculo::diff::vetorial::derivada_numerica(10e-6, &f_exemplo);
    let mut algoritmo = DescidaGrad{
        f: &f_exemplo, 
        df: &df_exemplo,
        alpha: 0.01,	
    };
    let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
    otimizacao.enumerate().for_each(|(n_passo, passo)| {println!("{:000} - {:?}", n_passo, passo); });
}

0 - Passo { x: VecStorage { data: [10.0, 15.0], nrows: Dyn(2), ncols: Const }, y: 584.0 }
1 - Passo { x: VecStorage { data: [9.769999999955417, 14.400000000036925], nrows: Dyn(2), ncols: Const }, y: 543.482900001122 }
2 - Passo { x: VecStorage { data: [9.5445999999788, 13.8240000000701], nrows: Dyn(2), ncols: Const }, y: 505.93914116340795 }
3 - Passo { x: VecStorage { data: [9.323708000019906, 13.271040000076937], nrows: Dyn(2), ncols: Const }, y: 471.14366023697903 }
4 - Passo { x: VecStorage { data: [9.107233840021536, 12.740198400094869], nrows: Dyn(2), ncols: Const }, y: 438.8887202844577 }
5 - Passo { x: VecStorage { data: [8.895089163255534, 12.230590464141642], nrows: Dyn(2), ncols: Const }, y: 408.9825649151375 }
6 - Passo { x: VecStorage { data: [8.687187379999841, 11.741366845583343], nrows: Dyn(2), ncols: Const }, y: 381.2481775203555 }
7 - Passo { x: VecStorage { data: [8.483443632409262, 11.271712171761692], nrows: Dyn(2), ncols: Const }, y: 355.5221373275744 }
8 - Passo 

64 - Passo { x: VecStorage { data: [1.6562157644355224, 1.1001456188915881], nrows: Dyn(2), ncols: Const }, y: 14.132338717204021 }
65 - Passo { x: VecStorage { data: [1.5930914491459802, 1.0561397941358663], nrows: Dyn(2), ncols: Const }, y: 13.54807724229468 }
66 - Passo { x: VecStorage { data: [1.5312296201621578, 1.013894202369734], nrows: Dyn(2), ncols: Const }, y: 12.994315917346338 }
67 - Passo { x: VecStorage { data: [1.4706050277581895, 0.9733384342748153], nrows: Dyn(2), ncols: Const }, y: 12.46926964621533 }
68 - Passo { x: VecStorage { data: [1.41119292720159, 0.934404896903871], nrows: Dyn(2), ncols: Const }, y: 11.971269282104428 }
69 - Passo { x: VecStorage { data: [1.3529690686564422, 0.8970287010286881], nrows: Dyn(2), ncols: Const }, y: 11.498753487648838 }
70 - Passo { x: VecStorage { data: [1.2959096872828724, 0.8611475529883705], nrows: Dyn(2), ncols: Const }, y: 11.050261195477926 }
71 - Passo { x: VecStorage { data: [1.2399914935363654, 0.8267016508692748], nrows

126 - Passo { x: VecStorage { data: [-0.5980481002985183, 0.087552111017839], nrows: Dyn(2), ncols: Const }, y: 2.5788479736624716 }
127 - Passo { x: VecStorage { data: [-0.6160871382923183, 0.08405002657695171], nrows: Dyn(2), ncols: Const }, y: 2.545430761027436 }
128 - Passo { x: VecStorage { data: [-0.6337653955264066, 0.0806880255138509], nrows: Dyn(2), ncols: Const }, y: 2.5133835049101703 }
129 - Passo { x: VecStorage { data: [-0.6510900876157777, 0.07746050449340736], nrows: Dyn(2), ncols: Const }, y: 2.4826482988569345 }
130 - Passo { x: VecStorage { data: [-0.668068285863086, 0.07436208431364832], nrows: Dyn(2), ncols: Const }, y: 2.453169816153724 }
131 - Passo { x: VecStorage { data: [-0.6847069201454481, 0.07138760094092866], nrows: Dyn(2), ncols: Const }, y: 2.424895185194923 }
132 - Passo { x: VecStorage { data: [-0.7010127817424028, 0.0685320969032599], nrows: Dyn(2), ncols: Const }, y: 2.397773871550929 }
133 - Passo { x: VecStorage { data: [-0.7169925261074717, 0.0657

()

153 - Passo { x: VecStorage { data: [-0.9772579684832561, 0.029079699485388133], nrows: Dyn(2), ncols: Const }, y: 2.0249504893585737 }
154 - Passo { x: VecStorage { data: [-0.9877128091133702, 0.02791651150589658], nrows: Dyn(2), ncols: Const }, y: 2.013996829175832 }
155 - Passo { x: VecStorage { data: [-0.9979585529307311, 0.026799851045575807], nrows: Dyn(2), ncols: Const }, y: 2.0034820786075356 }
156 - Passo { x: VecStorage { data: [-1.0079993818719135, 0.02572785700359681], nrows: Dyn(2), ncols: Const }, y: 1.9933884534904143 }
157 - Passo { x: VecStorage { data: [-1.0178393942346187, 0.024698742723359146], nrows: Dyn(2), ncols: Const }, y: 1.9836989055362693 }
158 - Passo { x: VecStorage { data: [-1.0274826063501141, 0.023710793014428688], nrows: Dyn(2), ncols: Const }, y: 1.9743970907124273 }
159 - Passo { x: VecStorage { data: [-1.0369329542232553, 0.022762361293824362], nrows: Dyn(2), ncols: Const }, y: 1.965467339067743 }
160 - Passo { x: VecStorage { data: [-1.046194295138

214 - Passo { x: VecStorage { data: [-1.3475673363929719, 0.0024106553807445863], nrows: Dyn(2), ncols: Const }, y: 1.7732473394530628 }
215 - Passo { x: VecStorage { data: [-1.350615989665469, 0.002314229165509829], nrows: Dyn(2), ncols: Const }, y: 1.7723262938568884 }
216 - Passo { x: VecStorage { data: [-1.3536036698727028, 0.002221659998844494], nrows: Dyn(2), ncols: Const }, y: 1.7714417570210412 }
217 - Passo { x: VecStorage { data: [-1.3565315964756497, 0.002132793598939031], nrows: Dyn(2), ncols: Const }, y: 1.770592280426897 }
218 - Passo { x: VecStorage { data: [-1.3594009645463512, 0.002047481854936528], nrows: Dyn(2), ncols: Const }, y: 1.7697764731343888 }
219 - Passo { x: VecStorage { data: [-1.3622129452557452, 0.0019655825807962657], nrows: Dyn(2), ncols: Const }, y: 1.7689929994848603 }
220 - Passo { x: VecStorage { data: [-1.3649686863510624, 0.0018869592775949684], nrows: Dyn(2), ncols: Const }, y: 1.7682405768963885 }
221 - Passo { x: VecStorage { data: [-1.3676693

274 - Passo { x: VecStorage { data: [-1.4546431819106953, 0.00020816567146475506], nrows: Dyn(2), ncols: Const }, y: 1.75205732761308 }
275 - Passo { x: VecStorage { data: [-1.4555503182729446, 0.0001998390446322773], nrows: Dyn(2), ncols: Const }, y: 1.751975854076924 }
276 - Passo { x: VecStorage { data: [-1.4564393119078733, 0.00019184548281980796], nrows: Dyn(2), ncols: Const }, y: 1.751897607156438 }
277 - Passo { x: VecStorage { data: [-1.4573105256701169, 0.0001841716634620738], nrows: Dyn(2), ncols: Const }, y: 1.751822459056965 }
278 - Passo { x: VecStorage { data: [-1.4581643151572266, 0.0001768047969008535], nrows: Dyn(2), ncols: Const }, y: 1.7517502870461361 }
279 - Passo { x: VecStorage { data: [-1.459001028854443, 0.00016973260495767306], nrows: Dyn(2), ncols: Const }, y: 1.7516809732533087 }
280 - Passo { x: VecStorage { data: [-1.459821008277693, 0.000162943300727747], nrows: Dyn(2), ncols: Const }, y: 1.7516144044768598 }
281 - Passo { x: VecStorage { data: [-1.460624

335 - Passo { x: VecStorage { data: [-1.4867738575261669, 1.725656383122498e-5], nrows: Dyn(2), ncols: Const }, y: 1.750174931440316 }
336 - Passo { x: VecStorage { data: [-1.4870383803760134, 1.6566301317411103e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501680041321614 }
337 - Passo { x: VecStorage { data: [-1.4872976127688986, 1.5903649273063536e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501613511472207 }
338 - Passo { x: VecStorage { data: [-1.4875516605139616, 1.5267503359339685e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501549616221528 }
339 - Passo { x: VecStorage { data: [-1.4878006273040345, 1.465680320222873e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501488251238186 }
340 - Passo { x: VecStorage { data: [-1.4880446147582749, 1.4070531060283997e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501429316322379 }
341 - Passo { x: VecStorage { data: [-1.488283722463466, 1.3507709826221515e-5], nrows: Dyn(2), ncols: Const }, y: 1.7501372715242298 }
342 - Passo { x: VecStorage { data: [-1.48851

## Método de Newton

## Definição

In [31]:
pub mod metodo_newton{
    use crate::{ferramentas::{calculo::diff::{escalar, matriz, vetorial}, exemplos}, monte_otimizacao, AlgoritmoOtimizacao, CriterioDeltaXMenorQueEpsilon, Matriz, Passo, Vetor};

    pub struct MetodoNewton<F, DF, D2F>{
        pub f  : F,
        pub df : DF,
        pub d2f: D2F 
    }

    impl<F, DF, D2F> AlgoritmoOtimizacao<f64, f64> for MetodoNewton<F, DF, D2F> 
    where 
        F   : Fn(&f64) -> f64,
        DF  : Fn(&f64) -> f64,
        D2F : Fn(&f64) -> f64,
    {
        fn proximo_passo(&mut self, passo: &Passo<f64, f64>) -> Passo<f64, f64> {
            let proximo_x = passo.x - ((self.df)(&passo.x) / (self.d2f)(&passo.x));
            let proximo_y = (self.f)(&proximo_x);
            Passo{x:proximo_x, y: proximo_y}
        }
    }
    impl< F, DF, D2F> AlgoritmoOtimizacao<Vetor, f64> for MetodoNewton<F, DF, D2F> 
    where 
        F :  Fn(&Vetor) -> f64,
        DF:  Fn(&Vetor) -> Vetor,
        D2F: Fn(&Vetor) -> Matriz,
    {
        fn proximo_passo(&mut self, passo: &Passo<Vetor, f64>) -> Passo<Vetor, f64> {
            let hessiana = (self.d2f)(&passo.x);
            let inverso_hessiana = hessiana.try_inverse().expect("a matriz hessiana não pode ser invertida. Isso é bem calvo da sua parte");
            let proximo_x = passo.x.clone() - (inverso_hessiana * (self.df)(&passo.x));
            let proximo_y = (self.f)(&proximo_x);
            Passo{x:proximo_x, y: proximo_y}
        }
    }
}

### Exemplos

#### No $\mathbb{R}^1$

In [35]:
{
    use metodo_newton::*;
    use crate::ferramentas::*;
    
    let (f_exemplo, passo_inicial) = exemplos::escalar::obtenha_f_e_passo_inicial();
    let df_exemplo  = calculo::diff::escalar::derivada_numerica(10e-4, &f_exemplo);
    let d2f_exemplo = calculo::diff::escalar::derivada_numerica(10e-4, &df_exemplo );

    let epsilon = 10e-5;
    let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

    let mut algoritmo = MetodoNewton{
        f: &f_exemplo, 
        df: &df_exemplo,
        d2f: &d2f_exemplo,
    };

    let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
    for passo_otim in otimizacao{
        println!("passo: {passo_otim:?}")
    }
}


passo: Passo { x: 15.0, y: 274.0 }
passo: Passo { x: -1.5000000416946477, y: 1.7500000000000018 }
passo: Passo { x: -1.500000000000222, y: 1.75 }


()

Que rápido!

#### Para $\mathbb{R}^n, n > 1$

In [38]:
{
    use metodo_newton::*;
    use crate::ferramentas::*;
    
    let (f_exemplo, passo_inicial) = exemplos::vetorial::obtenha_f_e_passo_inicial();
    let df_exemplo = calculo::diff::vetorial::derivada_numerica(1e-6, &f_exemplo);
    let d2f_exemplo = calculo::diff::matriz::derivada_numerica(1e-6,  &df_exemplo);

    let epsilon = 1e-5;
    let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

    let mut algoritmo = MetodoNewton{
        f: &f_exemplo, 
        df: &df_exemplo,
        d2f: &d2f_exemplo,
    };

    let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
    for passo_otim in otimizacao{
        println!("passo: {passo_otim:?}")
    }
}


passo: Passo { x: VecStorage { data: [10.0, 15.0], nrows: Dyn(2), ncols: Const }, y: 584.0 }
passo: Passo { x: VecStorage { data: [-1.5605793714285703, -0.07901659999999922], nrows: Dyn(2), ncols: Const }, y: 1.7661571063938004 }
passo: Passo { x: VecStorage { data: [-1.5000020225526336, 6.513981994360485e-7], nrows: Dyn(2), ncols: Const }, y: 1.7500000000049396 }
passo: Passo { x: VecStorage { data: [-1.5000000000133564, 6.265160881270954e-12], nrows: Dyn(2), ncols: Const }, y: 1.75 }


()

Note que o y da resposta é $ \approx 6.265 \times 10^{-12}$, indicando que seu valor é bem próximo de 0,

## Método de quase-newton
### Definição

In [42]:
pub mod metodo_quase_newton{
    use nalgebra::DMatrix;
    use crate::{ferramentas::{calculo::diff::{self, escalar::derivada_numerica}, exemplos}, monte_otimizacao, AlgoritmoOtimizacao, CriterioDeltaXMenorQueEpsilon, Matriz, Passo, PassoEscalar, Vetor};


    pub struct MetodoQuaseNewton< F, Df> 
        where F: Fn(&Vetor) -> f64, Df: Fn(&Vetor) -> Vetor 
    {
        pub f				: F,
        pub df				: Df,
        pub alfa			: f64,
        pub inv_hess 		: Matriz,
        pub df_x_anterior	: Vetor,
    }

    impl<F, DF> AlgoritmoOtimizacao<Vetor, f64> for MetodoQuaseNewton< F, DF> 
    where 
        F : Fn(&Vetor) -> f64,
        DF: Fn(&Vetor) -> Vetor,
    {
        fn proximo_passo(&mut self, passo_anterior: &Passo<Vetor, f64>) -> Passo<Vetor, f64> {
            let delta_x = - self.alfa * self.inv_hess.clone() * (self.df)(&passo_anterior.x);	
            let x = passo_anterior.x.clone() + delta_x.clone();
            let y = (self.f)(&x);
            let df_x = (self.df)(&x);
            self.inv_hess = {
                let yk: Vetor =  df_x.clone() - self.df_x_anterior.clone() ;
                let ident_matrix = Matriz::identity(yk.len(), yk.len());
                let yk_t_delta_x = (yk.transpose() * delta_x.clone())[0];
                let inv_hess = 
                    ( 	( 	( ident_matrix.clone() - ( (delta_x.clone() * yk.transpose()) / (yk_t_delta_x)) ) 
                        * 	self.inv_hess.clone()
                        * 	( ident_matrix.clone() - ( (yk * delta_x.clone().transpose()) / (yk_t_delta_x) ) ) 
                        )
                        + ((delta_x.clone() * delta_x.clone().transpose()) / yk_t_delta_x )
                    );
                inv_hess
            };
            self.df_x_anterior = df_x;
            Passo{x, y}
        }
    }
}

In [46]:
{
    use metodo_quase_newton::*;
    use crate::ferramentas::*;
    
	let epsilon = 10e-5;
	let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

	let (f_exemplo, passo_inicial) = exemplos::vetorial::obtenha_f_e_passo_inicial();
	let df_exemplo = calculo::diff::vetorial::derivada_numerica(10e-6, &f_exemplo);
	let primeira_hessiana  = calculo::diff::matriz::derivada_numerica(10e-6, &df_exemplo)(&passo_inicial.x);

	let mut algoritmo = MetodoQuaseNewton{
		f: &f_exemplo, 
		df: &df_exemplo,
		alfa: 0.5,	
		inv_hess: primeira_hessiana.try_inverse().unwrap(),
		df_x_anterior : df_exemplo(&passo_inicial.x)
	};
	let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
	otimizacao.enumerate().for_each(|(n_passo, passo)| {println!("{:000} - {:?}", n_passo, passo); });
}

0 - Passo { x: VecStorage { data: [10.0, 15.0], nrows: Dyn(2), ncols: Const }, y: 584.0 }
1 - Passo { x: VecStorage { data: [4.248219340848372, 7.499192209148075], nrows: Dyn(2), ncols: Const }, y: 147.26779316999767 }
2 - Passo { x: VecStorage { data: [1.3741096933282986, 3.749596096306274], nrows: Dyn(2), ncols: Const }, y: 38.12944830015418 }
3 - Passo { x: VecStorage { data: [-0.06294515352491747, 1.8747980478791861], nrows: Dyn(2), ncols: Const }, y: 10.844862072440737 }
4 - Passo { x: VecStorage { data: [-0.7814725767738901, 0.937399023932096], nrows: Dyn(2), ncols: Const }, y: 4.023715518065646 }
5 - Passo { x: VecStorage { data: [-1.140736288353967, 0.46869951198075516], nrows: Dyn(2), ncols: Const }, y: 2.31842887956768 }
6 - Passo { x: VecStorage { data: [-1.320368144188415, 0.2343497559939828], nrows: Dyn(2), ncols: Const }, y: 1.892107219891193 }
7 - Passo { x: VecStorage { data: [-1.410184072105639, 0.11717487799782106], nrows: Dyn(2), ncols: Const }, y: 1.7855268049711335

()

## Método dos gradientes conjugados

### Definição

In [48]:
pub mod grads_conjugados{
    use crate::{ferramentas::{calculo::diff, exemplos}, monte_otimizacao, AlgoritmoOtimizacao, CriterioDeltaXMenorQueEpsilon, Matriz, Passo, Vetor};


    pub struct GradConjugado<F, Df>{
        pub f : F,
        pub df: Df,
        pub Q : Matriz,
        pub d : Vetor
    }

    impl<F, Df> AlgoritmoOtimizacao<Vetor, f64> for GradConjugado<F, Df> where F: Fn(&Vetor) -> f64, Df: Fn(&Vetor) -> Vetor{
        fn proximo_passo(&mut self, passo_anterior: &Passo<Vetor, f64>) -> Passo<Vetor, f64> {
            let q_d = self.Q.clone() * self.d.clone();
            let q_d_dot_d = q_d.dot(&self.d);
            let alfa =     q_d_dot_d / q_d.dot(&q_d);
            let x_novo = passo_anterior.x.clone() + alfa * self.d.clone();
            let dy = (self.df)(&x_novo);
            let beta = q_d.dot(&dy) / q_d_dot_d;
            let d_novo = -dy + beta * self.d.clone();
            self.d = d_novo;
            let y_novo = (self.f)(&x_novo);
            Passo{x:x_novo, y:y_novo}
        }
    }
}

## Exemplo

In [54]:
{
    use ferramentas::*;
    use grads_conjugados::*;
        let epsilon = 1e-5;
        let criterio_parada = CriterioDeltaXMenorQueEpsilon(epsilon);

        let (f_exemplo, passo_inicial) = exemplos::vetorial::obtenha_f_e_passo_inicial();
        let df_exemplo = calculo::diff::vetorial::derivada_numerica(1e-6, &f_exemplo);
        let matriz_hessiana_diagonal =  { // usamos apenas a primeira.
            let hessiana: Matriz = (calculo::diff::matriz::derivada_numerica(10e-6, &df_exemplo)(&passo_inicial.x));
            hessiana
        }; 
        let algoritmo = GradConjugado{
            f: &f_exemplo, 
            df: &df_exemplo,
            Q: matriz_hessiana_diagonal,	
            d: df_exemplo(&passo_inicial.x)
        };
        let otimizacao = monte_otimizacao(algoritmo, criterio_parada, passo_inicial);
        otimizacao.enumerate().for_each(|(n_passo, passo)| {println!("{:000} - {:?}", n_passo, passo); });
}

0 - Passo { x: VecStorage { data: [10.0, 15.0], nrows: Dyn(2), ncols: Const }, y: 584.0 }
1 - Passo { x: VecStorage { data: [15.957928439583553, 30.5424220457544], nrows: Dyn(2), ncols: Const }, y: 2172.208354243589 }
2 - Passo { x: VecStorage { data: [21.15410249852396, 29.54534384039329], nrows: Dyn(2), ncols: Const }, y: 2260.813045307761 }
3 - Passo { x: VecStorage { data: [9.465287847447081, -1.0690031211558129], nrows: Dyn(2), ncols: Const }, y: 124.27307292345239 }
4 - Passo { x: VecStorage { data: [-0.8204038969923868, 0.8968379119074181], nrows: Dyn(2), ncols: Const }, y: 3.82048734369205 }
5 - Passo { x: VecStorage { data: [-1.173518027036582, -0.03171513059832265], nrows: Dyn(2), ncols: Const }, y: 1.8586021776878232 }
6 - Passo { x: VecStorage { data: [-1.4799153791535473, 0.02661101485158464], nrows: Dyn(2), ncols: Const }, y: 1.7518196842174085 }
7 - Passo { x: VecStorage { data: [-1.49034835679016, -0.000934000554731855], nrows: Dyn(2), ncols: Const }, y: 1.7500948989307

()